In [37]:
# Suzan Iloglu, Dec 11,2020
# Positive results from Pfizer, Moderna and AstraZeneca show a vaccine can work
# In this research, we wanna show the allocation of vaccine over counties given
# the number of total vaccines ordered by each state\


# Import packages
import csv
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image
import sodapy
from sodapy import Socrata
from collections import Counter
from scipy import stats


# MAPPING THE VACCINE ALLOCATION
The project presents multiple options for how to distribute vaccine within each county per state. It shows that what you choose to prioritize greatly impacts where vaccine would be sent. The followings are our options to choose to define vulnerability:

- Adult population

An extension of the federal government's vaccine distribution rationale from the state level to the county.

- Phase 1a

ACIP’s prioritization of healthcare personnel & long-term care facility residents

    - Phase 1a weighted by SVI (incl. race/ethnicity)
    
    Sub-allocation within Phase 1a by CDC’s Social Vulnerability Index 
    
    - Phase 1a weighted by SVI (excl. race/ethnicity)
    
	Sub-allocation within Phase 1a by CDC’s Social Vulnerability Index


### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2015-2019 (5-year) data for the following estimates:
<img src="Input/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [38]:
## import svi data downloaded from CDC website as cited above
svi_counties =  pd.read_csv('Input/SVI_2019_State_Data.csv')
svi_counties.head(5)

,FIPS,COUNTY,STATE,E_NOHSDP,E_AGE17,E_POV,E_PCI,E_GROUPQ,E_AGE65,EP_AGE65,E_TOTPOP,EP_NOHSDP,EP_POV,E_UNINSUR,EP_UNINSUR,E_HH,E_DISABL,EP_DISABL,E_UNEMP,EP_UNEMP,E_HU,E_MOBILE,EP_MOBILE,E_NOVEH,EP_NOVEH,M_NOHSDP,M_AGE17,M_POV,M_PCI,M_GROUPQ,M_AGE65,MP_AGE65,M_TOTPOP,MP_NOHSDP,MP_POV,M_UNINSUR,MP_UNINSUR,M_HH,M_DISABL,MP_DISABL,M_UNEMP,MP_UNEMP,M_HU,M_MOBILE,MP_MOBILE,M_NOVEH,MP_NOVEH,ST,EP_PCI,MP_PCI,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_CROWD,M_CROWD,EP_AGE17,MP_AGE17,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_CROWD,MP_CROWD,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,SPL_THEME3_xMin,RPL_THEME3_xMin,SPL_THEMES_xMin,RPL_THEMES_xMin,RPL_rank,RPL_xMin_rank
0,1063,Greene County,Alabama,1204,1884,3140,14884,68,1773,21.3,8324,20.8,38.1,1057,12.8,2951,2039,24.7,311,11.5,5112,1828,35.8,531,18.0,241,13.0,537,1570,32.0,NaN,NaN,NaN,4.2,6.5,260,3.1,246,318,3.8,137,5.1,29,206,4.0,177,5.8,1,14884,1570,333,107.154095,6889,NaN,4,66.452991,299,153.287312,64,51.244512,22.633349,NaN,11.284310,3.507149,82.760692,NaN,0.051640,0.857901,5.848983,2.998395,2.168756,1.727077,0.816915,NaN,0.994906,0.954955,0.972662,0.888889,3.811412,0.985673,0.751476,0.554919,0.960840,0.987902,3.255137,0.993314,0.986310,0.076421,1.062731,0.542502,0.751476,0.975148,0.584654,0.985399,0.110793,3.407469,0.920089,11.536749,0.994269,0.076421,0.076421,10.550439,0.992677,67.0,67.0
1,1005,Barbour County,Alabama,4812,5307,6875,18473,2886,4710,18.6,25361,26.8,30.7,2544,11.3,9345,4806,21.4,849,9.2,12013,3520,29.3,950,10.2,333,27.0,558,942,339.0,38.0,0.1,NaN,1.8,2.4,342,1.5,313,369,1.6,176,1.9,143,255,2.0,184,2.0,1,18473,942,444,105.801701,13743,NaN,454,170.569634,208,86.023253,360,157.175062,20.925831,NaN,4.751204,1.120935,54.189504,NaN,1.890721,0.710349,1.731458,0.715788,3.852327,1.676960,11.379677,NaN,0.976441,0.913327,0.939733,0.972939,3.802439,0.984400,0.505747,0.328558,0.882840,0.274117,1.991261,0.475326,0.896530,0.736564,1.633094,0.876472,0.335508,0.930724,0.863312,0.894377,0.947787,3.971708,0.995861,11.398502,0.992359,0.736564,0.736564,10.501972,0.991722,66.0,66.0
2,1107,Pickens County,Alabama,2680,4031,4204,23024,1829,3739,18.5,20243,18.5,22.7,1697,9.1,7637,4657,25.1,802,10.3,9588,2414,25.2,787,10.3,273,NaN,561,1440,264.0,23.0,0.1,NaN,1.9,3.0,326,1.7,279,383,2.1,204,2.5,116,254,2.7,170,2.1,1,23024,1440,535,113.877127,9383,NaN,322,132.582050,158,69.462220,100,63.126856,19.913056,NaN,7.005369,1.468997,46.351825,NaN,1.678307,0.691034,1.647893,0.724196,1.309415,0.825207,9.035222,NaN,0.885387,0.940043,0.773221,0.814072,3.412724,0.922954,0.496738,0.225406,0.966253,0.829672,2.518069,0.786692,0.857052,0.709848,1.566900,0.846864,0.320596,0.886300,0.277726,0.896552,0.918179,3.299353,0.886979,10.797046,0.974530,0.709848,0.709848,9.939994,0.973575,64.0,65.0
3,1131,Wilcox County,Alabama,1675,2577,3111,16841,251,2026,19.0,10681,23.5,30.1,1134,10.9,3854,2001,19.2,541,15.3,5777,2333,40.4,600,15.6,204,NaN,417,1299,108.0,NaN,NaN,NaN,2.9,4.1,221,2.1,213,272,2.6,152,3.8,44,196,3.4,131,3.1,1,16841,1299,218,75.802375,7814,NaN,65,115.585466,8,23.021729,54,55.713553,24.126954,NaN,5.656461,1.941846,73.157944,NaN,0.649351,NaN,0.138480,0.398505,1.401142,1.443528,2.349967,NaN,0.972620,0.979807,0.958993,0.939191,3.850612,0.990767,0.548618,0.758994,0.779370,0.553327,2.640308,0.841452,0.972620,0.453557,1.426177,0.767590,0.036347,0.990991,0.316869,0.974837,0.579752,2.898795,0.701687,10.815893,0.975486,0.453557,0.453557,9.843272,0.967845,65.0,64.0
4,1105,Perry County,Alabama,1010,2007,2612,15055,810,1772,19.1,9293,17.2,30.8,846,9.3,3070,2016,22.1,290,10.1,4736,1153,24.3,399,13.0,277,NaN,669,2347,200.0,NaN,NaN,NaN,4.7,7.8,283,3.1,314,337,3.7,184,6.2,75,251,5.3,180,5.6,1,15055,2347,225,113.225439,6566,

In [39]:
## Create the list for State
S = svi_counties.STATE.unique().tolist()
if "0" in S:
    S.remove(0)
State = [str(s).strip() for s in S]

In [40]:
## Replacing -999 values with 0 for calculations

svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)


In [41]:
# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]


In [42]:
cartesian_pro_county_state = [(i,county_of_states[i]) for i in location ]


In [43]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI/1000))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))

In [44]:

df_a = pd.read_csv("Input/ACSST5Y2019.S0101_data_with_overlays_2020-12-15T094007.csv", header=[1])
df_a.head(1)
df_a = df_a.rename(columns = {"Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 years and over":'Adult_pop'})
#Adult population
Adult_pop_county = dict(zip(df_a['FIPS'], df_a["Adult_pop"]))


In [45]:
Adult_pop_county

{1001: 42175,
 1003: 166595,
 1005: 20054,
 1007: 17862,
 1009: 44292,
 1011: 8120,
 1013: 15373,
 1015: 89666,
 1017: 26595,
 1019: 20658,
 1021: 33420,
 1023: 10322,
 1025: 18841,
 1027: 10559,
 1029: 11533,
 1031: 39403,
 1033: 43218,
 1035: 9790,
 1037: 8956,
 1039: 29037,
 1041: 10738,
 1043: 64148,
 1045: 37892,
 1047: 29720,
 1049: 53800,
 1051: 62915,
 1053: 28769,
 1055: 80579,
 1057: 13014,
 1059: 23582,
 1061: 20599,
 1063: 6440,
 1065: 11355,
 1067: 13537,
 1069: 80349,
 1071: 40870,
 1073: 508503,
 1075: 10835,
 1077: 74247,
 1079: 25875,
 1081: 126764,
 1083: 73144,
 1085: 7808,
 1087: 15460,
 1089: 282589,
 1091: 14918,
 1093: 23592,
 1095: 71756,
 1097: 316303,
 1099: 16522,
 1101: 173501,
 1103: 91846,
 1105: 7286,
 1107: 16212,
 1109: 26863,
 1111: 17751,
 1113: 43783,
 1115: 67948,
 1117: 162807,
 1119: 10327,
 1121: 63130,
 1123: 32072,
 1125: 163687,
 1127: 49854,
 1129: 12753,
 1131: 8104,
 1133: 18858,
 2013: 2901,
 2016: 4716,
 2020: 221655,
 2050: 11740,
 2060:

# Phase 1a population
ACIP’s prioritization of healthcare personnel & long-term care facility residents

In [46]:
#Phase 1a population includes the number of health care workers and long-term care facility residents
first_phase = pd.read_csv("Input/Phase_1a_pop.csv")
first_phase = first_phase.fillna(0)
#Set FIPS type as int
first_phase['FIPS'] = first_phase['FIPS'].astype(int)


In [47]:
first_phase.head(80)

,FIPS,NAME,hp_emp,ltcf_res,phase_1a
0,2013,"Aleutians East Borough, Alaska",57,0,57
1,2016,"Aleutians West Census Area, Alaska",148,0,148
2,2060,"Bristol Bay Borough, Alaska",17,0,17
3,2068,"Denali Borough, Alaska",20,0,20
4,2070,"Dillingham Census Area, Alaska",418,0,418
5,2100,"Haines Borough, Alaska",185,0,185
6,2105,"Hoonah-Angoon Census Area, Alaska",70,0,70
7,2158,"Kusilvak Census Area, Alaska",190,0,190
8,2164,"Lake and Peninsula Borough, Alaska",33,0,33
9,2185,"North Slope Borough, Alaska",287,0,287


In [48]:
#Create a dictionary for the Phase 1a population
Firstphase_county = dict(zip(first_phase.FIPS, first_phase.phase_1a))

In [49]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

In [50]:
Firstphase_county

{2013: 57,
 2016: 148,
 2060: 17,
 2068: 20,
 2070: 418,
 2100: 185,
 2105: 70,
 2158: 190,
 2164: 33,
 2185: 287,
 2198: 251,
 2230: 20,
 2240: 196,
 2282: 9,
 2290: 180,
 4011: 170,
 4012: 460,
 4023: 1625,
 5013: 210,
 6003: 29,
 6051: 441,
 6091: 142,
 8019: 357,
 8023: 147,
 8027: 70,
 8033: 48,
 8047: 365,
 8049: 565,
 8053: 17,
 8057: 31,
 8061: 98,
 8065: 159,
 8079: 28,
 8091: 159,
 8093: 453,
 8097: 692,
 8109: 240,
 8111: 13,
 8113: 317,
 8117: 1092,
 12043: 159,
 12077: 204,
 12125: 381,
 13003: 196,
 13007: 89,
 13011: 689,
 13053: 221,
 13085: 1048,
 13101: 97,
 13181: 275,
 13191: 579,
 13209: 481,
 13229: 852,
 13231: 781,
 13239: 69,
 13249: 280,
 13263: 345,
 13265: 24,
 13307: 104,
 15005: 7,
 16003: 119,
 16015: 258,
 16023: 93,
 16025: 36,
 16033: 4,
 16037: 137,
 16051: 1129,
 16053: 1007,
 16061: 194,
 16081: 460,
 17003: 334,
 17151: 135,
 17153: 330,
 17155: 315,
 18121: 521,
 20049: 144,
 20075: 113,
 20083: 129,
 20089: 127,
 20101: 84,
 20107: 457,
 22023: 2

Since we allocate tha vaccine proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [51]:
# This function return the value for the state for the given dictionary

# More specifically sum up the values for the counties of each state

def total_state(dict_1):
    state_dict = {}

    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for j in dict_1 if (j,s) in cartesian_pro_county_state)  
    return state_dict



In [52]:
Adult_pop_state = total_state(Adult_pop_county)


In [53]:
Adult_pop_state

{'Alabama': 3779874.0,
 'Alaska': 552674.0,
 'Arizona': 5414955.0,
 'Arkansas': 2295102.0,
 'California': 30261351.0,
 'Colorado': 4349344.0,
 'Connecticut': 2831241.0,
 'Delaware': 753564.0,
 'District of Columbia': 568753.0,
 'Florida': 16719174.0,
 'Georgia': 7898607.0,
 'Hawaii': 1117456.0,
 'Idaho': 1276603.0,
 'Illinois': 9879105.0,
 'Indiana': 5093212.0,
 'Iowa': 2410771.0,
 'Kansas': 2199582.0,
 'Kentucky': 3439746.0,
 'Louisiana': 3561012.0,
 'Maine': 1082994.0,
 'Maryland': 4677166.0,
 'Massachusetts': 5479293.0,
 'Michigan': 7787387.0,
 'Minnesota': 4267530.0,
 'Mississippi': 2270925.0,
 'Missouri': 4723298.0,
 'Montana': 822263.0,
 'Nebraska': 1440464.0,
 'Nevada': 2291340.0,
 'New Hampshire': 1088160.0,
 'New Jersey': 6916982.0,
 'New Mexico': 1603978.0,
 'New York': 15463820.0,
 'North Carolina': 7968262.0,
 'North Dakota': 580973.0,
 'Ohio': 9050387.0,
 'Oklahoma': 2975906.0,
 'Oregon': 3261860.0,
 'Pennsylvania': 10129139.0,
 'Rhode Island': 849887.0,
 'South Carolina':

In [54]:
Firstphase_State = total_state(Firstphase_county)


In [55]:
vac_pf = pd.read_csv('https://data.cdc.gov/resource/saz5-9hgg.csv')
vac_mo = pd.read_csv('https://data.cdc.gov/resource/b7pe-5nws.csv')

vac_pf['jurisdiction'] = vac_pf['jurisdiction'].str.replace("*", "")
vac_pf.dropna(subset = ['total_pfizer_allocation_first_dose_shipments'], inplace=True)
vac_pf['first_doses'] = vac_pf['total_pfizer_allocation_first_dose_shipments'].str.replace(",", "")
vac_pf['first_doses'] = vac_pf['first_doses'].astype(int)

vac_mo['jurisdiction'] = vac_mo['jurisdiction'].str.replace("*", "")
vac_mo.dropna(subset = ['total_allocation_moderna_second_dose_shipments'], inplace=True)
vac_mo['first_doses'] = vac_mo['total_allocation_moderna_second_dose_shipments'].str.replace(",", "")
vac_mo['first_doses'] = vac_mo['first_doses'].astype(int)

vac_mo.head(5)



,jurisdiction,hhs_region,doses_allocated_week_12_21,second_dose_shipment_12_21,doses_allocated_week_12_28,second_dose_shipment_12_28,doses_allocated_week_01_04,second_dose_shipment_01_04,doses_allocated_week_of_01_10,second_dose_shipment_week_of_01_10,doses_distribution_week_of_01_18,second_dose_week_of_01_18,doses_allocated_week_of_01_25,second_dose_week_of_01_25,doses_allocated_week_of_02_01,second_dose_shipment_week_of_02_01,total_moderna_allocation_first_dose_shipments,total_allocation_moderna_second_dose_shipments,first_doses
0,Connecticut,Region 1,"63,300","63,300","21,900","21,900","21,800","21,800","21,900","21,900","23,000","23,000","23,000","23,000","30,300","30,300","205,200","205,200",205200
1,Maine,Region 1,"24,200","24,200","8,400","8,400","8,300","8,300","8,400","8,400","8,800","8,800","8,800","8,800","11,600","11,600","78,500","78,500",78500
2,Massachusetts,Region 1,"121,900","121,900","42,100","42,100","42,000","42,000","42,100","42,100","44,300","44,300","44,300","44,300","58,400","58,400","395,100","395,100",395100
3,New Hampshire,Region 1,"24,200","24,200","8,400","8,400","8,400","8,400","8,400","8,400","8,800","8,800","8,800","8,800","11,600","11,600","78,600","78,600",78600
4,Rhode Island,Region 1,"19,000","19,000","6,600","6,600","6,600","6,600","6,600","6,600","6,900","6,900","6,900","6,900","9,100","9,100","61,700","61,700",61700


In [56]:

Jur_state = {'New York City': 'New York', 'Philadelphia': 'Pennsylvania', 'Chicago':'Illinois'}

In [57]:

Vaccine_budget_state = {s:0 for s in State}

Vaccine_budget_st =  dict(zip(vac_pf.jurisdiction, vac_pf.first_doses + vac_mo.first_doses) )

sum_v = 0
for s in Vaccine_budget_st:
    
    if s in State:
        Vaccine_budget_state[s] = Vaccine_budget_st[s]
        sum_v += Vaccine_budget_st[s]
    
    if s in Jur_state:    
        Vaccine_budget_state[Jur_state[s]] += Vaccine_budget_st[s]
        sum_v += Vaccine_budget_st[s]
        
print (sum_v)

35732925


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We use the estimate percentage of these variables in a county base. To calculate the SVI value for each county, we take the average of the estimate percentage of these 15 variables.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} c^k_j$


In [58]:
# Sum all SVI variable estimated percentage values for each county

SVI_county_sum = dict(Counter(E_POV) + Counter(E_UNEMP) + Counter(E_PCI) + Counter(E_NOHSDP) + Counter(E_AGE65) + Counter(E_AGE17) + Counter(E_DISABL) + Counter(E_SNGPNT) + Counter(E_MINRTY) + Counter(E_LIMENG) + Counter(E_MUNIT) + Counter(E_MOBILE) + Counter(E_CROWD) + Counter(E_NOVEH) + Counter(E_GROUPQ))

# Divide the sum of all SVI variable values
SVI_county = {j: (SVI_county_sum[j]/(15*100)) for j in SVI_county_sum }


# SVI calculation (excl. race/ethnicity)

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We use the estimate percentage of these variables in a county base. To calculate the SVI value for each county, we take the average of the estimate percentage of these 14 (Minority excluded) variables.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{14}\sum_{k \in K} c^k_j$

In [59]:
# Sum all SVI variable estimate percentage values (except Minortity) for each county

SVI_county_sum_no_race = dict(Counter(E_POV) + Counter(E_UNEMP) + Counter(E_PCI) + Counter(E_NOHSDP) + Counter(E_AGE65) + Counter(E_AGE17) + Counter(E_DISABL) + Counter(E_SNGPNT) + Counter(E_LIMENG) + Counter(E_MUNIT) + Counter(E_MOBILE) + Counter(E_CROWD) + Counter(E_NOVEH) + Counter(E_GROUPQ))


# Divide the sum of all SVI variable values
SVI_county_no_race = {j: (SVI_county_sum_no_race[j]/(14*100)) for j in SVI_county_sum }


# Proportional Vaccine Allocation

We consider allocating vaccines ordered by each state to its counties proportional to multiple variables. 

- Adult population

An extension of the federal government's vaccine distribution rationale from the state level to the county.

- Phase 1a

ACIP’s prioritization of healthcare personnel & long-term care facility residents

    - Phase 1a weighted by SVI (incl. race/ethnicity)
    
    Sub-allocation within Phase 1a by CDC’s Social Vulnerability Index 
    
    - Phase 1a weighted by SVI (excl. race/ethnicity)
    
	Sub-allocation within Phase 1a by CDC’s Social Vulnerability Index


To calculate the total number of allocated vaccines to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the total amount of vaccine for each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of vaccines allocated to each county for the chosen vulnerability criteria as values. 

<a href="https://covid.cdc.gov/covid-data-tracker/#vaccinations">CDC website</a>  provides the total number of distributed vaccine in each state. 


In [60]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
    

    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            
            prop_allocate[j,s] = float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

SyntaxError: invalid syntax (<ipython-input-60-d46fc10b4229>, line 8)

In [ ]:
def total_state_pop(dict_1):
    state_dict = {}

    for s in State:
        state_dict [s] = sum(float(dict_1[j]*Firstphase_county[j]) for j in dict_1 if (j,s) in cartesian_pro_county_state)  
    return state_dict



In [ ]:
def Proportional_allocation_pop(county_level, state_budget):
    prop_allocate = {}
    state_level = total_state_pop(county_level)
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            
            prop_allocate[j,s] = min (((float(county_level[j])*Firstphase_county[j])/(float(state_level[s])))*float(state_budget[s]), Firstphase_county[j])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

# Proportional allocation for different vulnerability values

Let V = {Adult population, Phase 1a population, SVI weighted Phase 1a population, SVI (excl. race/ethnicity) weighted Phase 1a population}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j. $Vac_s$ represents the total number of vaccine state s ordered. 

$Prop_{v_j} = \frac{v_j}{v_s}*Vac_s$



In [ ]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to SVI score in each county
Proportional_to_Adult_pop = Proportional_allocation(Adult_pop_county, Adult_pop_state, Vaccine_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_Firstphase = Proportional_allocation(Firstphase_county, Firstphase_State, Vaccine_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation_pop(SVI_county, Vaccine_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_SVI_no_race = Proportional_allocation_pop(SVI_county_no_race, Vaccine_budget_state)


In [ ]:
for (j,s) in cartesian_pro_county_state:
    print (j,Firstphase_county[j], Firstphase_State[s], Vaccine_budget_state[s])

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [ ]:
# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])  
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [ ]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [ ]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [ ]:
Adult_pop_county = order_k(Adult_pop_county)
Firstphase_county = order_k(Firstphase_county)
SVI_county = order_k(SVI_county)
SVI_county_no_race = order_k(SVI_county_no_race)

In [ ]:
# Write file allocation with each strategies for each county 

Strategies = ["Adult_pop", "Firstphase", "SVI", "SVI_no_race"]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_vaccine_allocation.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_Adult_pop.values(),        Adult_pop_county.values(),     percentile_ranks(Adult_pop_county).values()
                   , Proportional_to_Firstphase.values(),       Firstphase_county.values(),    percentile_ranks(Firstphase_county).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_SVI_no_race.values(),      SVI_county_no_race.values(),   percentile_ranks(SVI_county_no_race).values()
                    ):                    
       
        writer.writerow(row)

In [ ]:
Vaccine_state = {s: Vaccine_budget_state[s] for s in State}

In [ ]:
Vaccine_state

In [ ]:
writefile = 'Output/State_level_vaccine_allocation.csv'

cl = ['State', 'Vaccine_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Vaccine_state.values()):
        writer.writerow(row)